In [2]:
# *_* coding:utf-8 *_*
import sys
import os
import tensorflow as tf 
import numpy as np
slim = tf.contrib.slim

#train_data_path = '/data/CVPR_Release/v2/dataset/training'

In [3]:
#def get_batch(dataset_dir,num_readers=2,batch_size=5,net=None,FLAGS=None,
 #               file_pattern = '*.tfrecord',is_training = True,shuffe = False):

In [3]:
#def get_batch():
    #Definimos la ruta de los tfrecords
    filename = 'C:\\Users\\rober\\Practicas\\training\\training_v2.tfrecords'
    #Función encargada de añadir a una cola FIFO la lista de ficheros, además el parametro epochs
    #nos proporciona el numero de tensores que obtendremos en cada iteracción
    filename_queue = tf.train.string_input_producer([filename], num_epochs=50)
    #Definimos un lector
    reader = tf.TFRecordReader()
    serialized_example = reader.read(filename_queue)
    #Diccionario con las características necesarias.Decodificamos el "record" leido
    #por el lector
    features = tf.parse_single_example(serialized_example,
                features={
                    'image/encoded': tf.FixedLenFeature([], tf.string, default_value=''),
                    'label/encoded': tf.FixedLenFeature([], tf.string, default_value=''),
                })  # Nos devuelve la imagen y la etiqueta
    #Convierte la etiqueta a su formato original 
    with tf.name_scope('cmc/label'):
        label = tf.decode_raw(features['label/encoded'], tf.float32)
        label = tf.transpose(tf.reshape(label, [3,1,240,240]), (0,2,3,1))
    #Convierte la imagen a su formato original 
    with tf.name_scope('cmc/image'):
        image = tf.decode_raw(features['image/encoded'], tf.float32)
        image = tf.transpose(tf.reshape(image, [3,4,240,240]), (0,2,3,1))
    #image = tf.reshape(image, [32, 100, 3])
    #Realiza el casteo a float32 e int32
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.int32)
    print("image", image.get_shape(), label.get_shape())
    
    ########REALIZAMOS EL PREPROCESADO AQUÍ################
    """
    if (np.random.choice([0, 1]) == 1):
        image = tf.image.flip_left_right(image)
        label = tf.image.flip_left_right(label)
    """
        
    do_a_crop_flip = tf.random_uniform([], seed=None)
    do_a_crop_flip = tf.greater(do_a_crop_flip, 0.5)
    #tf.cond es evaluada en tiempo de ejecución mientras que if-els
    #es evaluada en el tiempo de contruccion del grafo vamos previamente a la ejecución
        #condicion,if = lambda: tf.reverse_v2,else = lambda: image/label
    image = tf.cond(do_a_crop_flip, lambda: tf.reverse_v2(image, [2]),lambda: image)
    label = tf.cond(do_a_crop_flip, lambda: tf.reverse_v2(label, [2]),lambda: label)
    #########FIN DEL PREPROCESADO##########################
    
    ## Creamos los batches por unos tensores barajeados aleatoriamente
    batch_size = 5
    sh_images, sh_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=1,
            capacity=100 * batch_size,
            min_after_dequeue=batch_size)

    #return sh_images, sh_labels

IndentationError: unexpected indent (<ipython-input-3-551ce50c9853>, line 3)

In [16]:
#def get_batch():
#Definimos la ruta de los tfrecords
filename = 'C:\\Users\\rober\\Practicas\\training\\training_v2.tfrecords'
with tf.Session() as sess:

    feature={
        'image/encoded': tf.FixedLenFeature([], tf.string,default_value=''),
        'label/encoded': tf.FixedLenFeature([], tf.string,default_value='')
    }
    #Función encargada de añadir a una cola FIFO la lista de ficheros, además el parametro epochs
    #nos proporciona el numero de tensores que obtendremos en cada iteracción
    filename_queue = tf.train.string_input_producer([filename], num_epochs=50)
    #Definimos un lector
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example, features=feature)
    #Diccionario con las características necesarias.Decodificamos el "record" leido
    #por el lector
    #features = tf.parse_single_example(serialized_example,)  
    # Nos devuelve la imagen y la etiqueta
    #Convierte la etiqueta a su formato original 
    with tf.name_scope('label'):
        label = tf.decode_raw(features['label/encoded'], tf.float32)
        label = tf.transpose(tf.reshape(label, [1,240,240]), (0,2,1))
    #Convierte la imagen a su formato original 
    with tf.name_scope('image'):
        image = tf.decode_raw(features['image/encoded'], tf.float32)
        image = tf.transpose(tf.reshape(image, [4,240,240]), (0,2,1))
        #image = tf.reshape(image, [32, 100, 3])
    #Realiza el casteo a float32 e int32
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.int32)
    print("image", image.get_shape(), label.get_shape())

    ########REALIZAMOS EL PREPROCESADO AQUÍ################      
    do_a_crop_flip = tf.random_uniform([], seed=None)
    do_a_crop_flip = tf.greater(do_a_crop_flip, 0.5)
    #tf.cond es evaluada en tiempo de ejecución mientras que if-els
    #es evaluada en el tiempo de contruccion del grafo vamos previamente a la ejecución
    #condicion,if = lambda: tf.reverse_v2,else = lambda: image/label
    image = tf.cond(do_a_crop_flip, lambda: tf.reverse_v2(image, [2]),lambda: image)
    label = tf.cond(do_a_crop_flip, lambda: tf.reverse_v2(label, [2]),lambda: label)
    #########FIN DEL PREPROCESADO##########################

    ## Creamos los batches por unos tensores barajeados aleatoriamente
    batch_size = 152
    sh_images, sh_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=1,
            capacity=100 * batch_size,
            min_after_dequeue=batch_size)
    print("images", sh_images.get_shape(), sh_labels.get_shape())
    sh_images = tf.transpose(tf.reshape(sh_images, [batch_size,4,240,240]), (1,2,3,0))
    sh_labels = tf.transpose(tf.reshape(sh_labels, [batch_size,1,240,240]), (1,2,3,0))
    print("sh_images",sh_images.get_shape(),sh_labels.get_shape())
    #print("images", sh_images.get_shape(), sh_labels.get_shape())
    #return sh_images, sh_labels

image (4, 240, 240) (1, 240, 240)
images (152, 4, 240, 240) (152, 1, 240, 240)
sh_images (4, 240, 240, 152) (1, 240, 240, 152)
